In [1]:
import pandas as pd
import pyarrow.parquet as pq 
train = pq.read_table(source='C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/train').to_pandas()
test = pq.read_table(source='C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/test').to_pandas()

In [2]:
for column in train:
    if(column == "content"):
        train[column].fillna(value="",inplace=True)
    else:
        if(train[column].isnull().sum() != 0):
            media = train[column].mean()
            train[column].fillna(value=media, inplace=True)

In [3]:
for column in test:
    if(column == "content"):
        test[column].fillna(value="",inplace=True)
    else:
        if(test[column].isnull().sum() != 0):
            media = test[column].mean()
            test[column].fillna(value=media, inplace=True)

In [4]:
train = train.iloc[:(8*len(train)//10)]
valid = train.iloc[(8*len(train)//10):]

In [5]:
X_train = train.drop(['url','timedelta','popular','shares'],axis = 1)
y_train = train["popular"]*1

X_valid = valid.drop(['url','timedelta','popular','shares'],axis = 1)
y_valid = valid["popular"]*1

X_test = test.drop(['url','timedelta','popular','shares'],axis = 1)
y_test = test["popular"]*1

In [6]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from category_encoders import TargetEncoder

In [7]:
ohe_surprise1 = OneHotEncoder()
encoded_surp = ohe_surprise1.fit_transform(X_train[["surprise1"]]).toarray()
encoded_surp_test = ohe_surprise1.transform(X_test[["surprise1"]]).toarray()
encoded_surp_valid = ohe_surprise1.transform(X_valid[["surprise1"]]).toarray()

feature_labels = np.array(ohe_surprise1.categories_).ravel()
X_train = X_train.reset_index().drop(['index','surprise1'],axis = 1).join(pd.DataFrame(encoded_surp, columns = feature_labels))
X_test = X_test.reset_index().drop(['index','surprise1'],axis = 1).join(pd.DataFrame(encoded_surp_test, columns = feature_labels))
X_valid = X_valid.reset_index().drop(['index','surprise1'],axis = 1).join(pd.DataFrame(encoded_surp_valid, columns = feature_labels))

In [8]:
vect_content = CountVectorizer(analyzer = "word", stop_words = "english", max_features = 15)
encoded_content = vect_content.fit_transform(X_train["content"].tolist()).toarray()
encoded_content_test = vect_content.transform(X_test["content"].tolist()).toarray()
encoded_content_valid = vect_content.transform(X_valid["content"].tolist()).toarray()

In [9]:
X_train = X_train.reset_index().drop(['index','content'],axis = 1).join(pd.DataFrame(encoded_content, columns = vect_content.get_feature_names_out()))
X_test = X_test.reset_index().drop(['index','content'],axis = 1).join(pd.DataFrame(encoded_content_test, columns = vect_content.get_feature_names_out()))
X_valid = X_valid.reset_index().drop(['index','content'],axis = 1).join(pd.DataFrame(encoded_content_valid, columns = vect_content.get_feature_names_out()))

In [10]:
te = TargetEncoder()
encoded_surp2 = te.fit_transform(X_train["surprise2"],y_train)
encoded_surp2_test = te.transform(X_test["surprise2"],y_test)
encoded_surp2_valid = te.transform(X_valid["surprise2"],y_valid.reset_index().drop(["index"],axis = 1))

X_train = X_train.reset_index().drop(['index','surprise2'],axis = 1).join(pd.DataFrame(encoded_surp2, columns = te.get_feature_names()))
X_test = X_test.reset_index().drop(['index','surprise2'],axis = 1).join(pd.DataFrame(encoded_surp2_test, columns = te.get_feature_names()))
X_valid = X_valid.reset_index().drop(['index','surprise2'],axis = 1).join(pd.DataFrame(encoded_surp2_valid, columns = te.get_feature_names()))

C:\Users\tomas\AppData\Local\Programs\Python\Python37\lib\site-packages\category_encoders\target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\Users\tomas\AppData\Local\Programs\Python\Python37\lib\site-packages\category_encoders\target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [33]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
param_grid = {'bootstrap': [True, False],
              'max_depth': [1,2,3,4],
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'n_estimators': [200, 400, 600, 800, 1000]}

In [34]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(estimator = model, param_distributions = param_grid, cv = 3, verbose = True, n_jobs = -1, random_state = 42)
rs.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 2, 3, 4],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000]},
                   random_state=42, verbose=True)

In [35]:
print(rs.best_score_)

0.793130038555906


In [36]:
modeloPrueba = rs.best_estimator_

In [37]:
modeloPrueba.fit(X_train,y_train)

RandomForestClassifier(max_depth=1, max_features='sqrt', min_samples_leaf=2,
                       n_estimators=1000, random_state=42)

In [38]:
print(f'Accuracy - : {modeloPrueba.score(X_train,y_train):.3f}')

Accuracy - : 0.793


In [39]:
prediccion_binaria = modeloPrueba.predict(X_valid)
prediccion_binaria


array([0, 0, 0, ..., 0, 0, 0])

In [40]:
proba_preds = modeloPrueba.predict_proba(X_valid)[:,1]
proba_preds

array([0.19775153, 0.19553239, 0.18065955, ..., 0.20711292, 0.19491697,
       0.21464108])

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_valid, prediccion_binaria))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      4586
           1       0.00      0.00      0.00      1120

    accuracy                           0.80      5706
   macro avg       0.40      0.50      0.45      5706
weighted avg       0.65      0.80      0.72      5706



C:\Users\tomas\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tomas\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tomas\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [42]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid,proba_preds)

0.7020092206093078

In [43]:
roc_auc_score(y_test,modeloPrueba.predict_proba(X_test)[:,1])

0.7123260719378623